In [ ]:
import os
import json
import scipy.io
import pandas as pd
from pathlib import Path

# Load Metadata
def load_metadata(filename):
    """
    Load the metadata file.

    Parameters:
        filename (str): Path to the metadata file.

    Returns:
        dict: Metadata for the matrices.
    """
    if not os.path.exists(filename):
        print(f"Metadata file {filename} not found.")
        return {}

    with open(filename, "r") as f:
        return {matrix["name"]: matrix for matrix in json.load(f)}

# Analyze Matrices
def analyze_matrices(matrix_dir, metadata_file):
    """
    Analyze all .mtx matrices in the directory and display their properties.

    Parameters:
        matrix_dir (str): Path to the directory containing .mtx files.
        metadata_file (str): Path to the metadata file for additional matrix info.

    Returns:
        pd.DataFrame: DataFrame containing matrix details.
    """
    # Load metadata
    metadata = load_metadata(metadata_file)

    # List to store matrix details
    matrix_details = []

    # Iterate over all .mtx files in the directory
    for mtx_file in Path(matrix_dir).glob("*.mtx"):
        matrix_name = mtx_file.stem

        try:
            # Load the matrix
            matrix = scipy.io.mmread(mtx_file)
            rows, cols = matrix.shape
            nnz = matrix.nnz

            # Calculate sparsity if metadata is unavailable
            sparsity = metadata.get(matrix_name, {}).get("sparsity", 1 - (nnz / (rows * cols)))

            # Retrieve additional metadata
            group = metadata.get(matrix_name, {}).get("group", "Unknown")
            kind = metadata.get(matrix_name, {}).get("kind", "Unknown")

            # Append to details list
            matrix_details.append({
                "Name": matrix_name,
                "Group": group,
                "Kind": kind,
                "Rows": rows,
                "Columns": cols,
                "Non-Zero Elements": nnz,
                "Sparsity": round(sparsity, 6),
            })
        except Exception as e:
            print(f"Error processing matrix {matrix_name}: {e}")

    # Convert details to a DataFrame
    return pd.DataFrame(matrix_details)

# Inputs
matrix_dir = "../ssmatrices"  # Directory containing .mtx files
metadata_file = "../ssmatrices/ss_matrix_metadata.json"  # Metadata file

# Analyze Matrices
matrix_df = analyze_matrices(matrix_dir, metadata_file)

# Display Matrices in Row-Wise Tabular Format
if not matrix_df.empty:
    from IPython.display import display
    display(matrix_df)
else:
    print("No matrices found to display.")
